### input - text of the tweet, and outputs are - label, an a demographic feature value

In [ ]:
import itertools
import spacy
import random
import os
from spacy.util import minibatch, compounding
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)
root_path = 'gdrive/My Drive/AI&Ethics/'
os.chdir(root_path)

In [ ]:
!pip -q install datasets
!pip -q install sentencepiece==0.1.94
# !pip install transformers==4.0.1
!pip -q install pytorch-lightning
!pip -q install demoji
!pip -q install tweet-preprocessor
!pip -q install transformers
!pip -q install ekphrasis

     |████████████████████████████████| 194kB 20.8MB/s 
     |████████████████████████████████| 245kB 41.5MB/s 
     |████████████████████████████████| 112kB 56.4MB/s 
     |████████████████████████████████| 1.1MB 19.4MB/s 
     |████████████████████████████████| 849kB 21.6MB/s 
     |████████████████████████████████| 184kB 53.2MB/s 
     |████████████████████████████████| 276kB 58.6MB/s 
     |████████████████████████████████| 829kB 56.5MB/s 
     |████████████████████████████████| 1.3MB 55.0MB/s 
     |████████████████████████████████| 296kB 56.2MB/s 
     |████████████████████████████████| 143kB 56.6MB/s 
     |████████████████████████████████| 2.1MB 17.6MB/s 
     |████████████████████████████████| 870kB 55.3MB/s 
     |████████████████████████████████| 3.3MB 54.4MB/s 
     |████████████████████████████████| 81kB 7.3MB/s 
     |████████████████████████████████| 184kB 26.9MB/s 
     |████████████████████████████████| 71kB 9.5MB/s 


In [ ]:
import numpy as np
import pandas as pd
import re
from transformers import AutoModel, AutoTokenizer
import json
import pickle
import torch.nn as nn
import torch
import copy
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, RandomSampler, SequentialSampler, random_split, DataLoader, IterableDataset, ConcatDataset
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score 
from tqdm import tqdm
import demoji 
import random
demoji.download_codes() 
import preprocessor as p
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from sklearn.metrics import classification_report, accuracy_score
from ekphrasis.dicts.emoticons import emoticons
plt.rcParams['figure.figsize'] = [15, 8]
plt.rcParams.update({'font.size': 8})
RANDOM_SEED = 42
model_path = 'ai4bharat/indic-bert'
model_path = 'bert-base-uncased'
annotator_file = "data/annotators_demography.csv"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

... OK (Got response in 0.16 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK


# Run these

In [ ]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)  
    torch.manual_seed(seed_value)  
    random.seed(seed_value)
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
random_seed(RANDOM_SEED, True)

In [ ]:
categories = ['Age', 'Country', 'Religion', 'Race', 'Gender']
categories_dict = {categories[i]: i for i in range(len(categories))}
label_dict = {}
dict_label = {}

In [ ]:
class Dataset():
    def __init__(self, data, batch_size = 32, train = False):
        self.data = data
        # self.val_data = val_data
        self.batch_size = batch_size                         
        self.count_dic = {}
        self.train = train
        self.inputs, self.labels, self.demographies = self.process_data(self.data)
        self.DataLoader = self.get_dataloader(self.inputs, self.labels, self.demographies)
        # self.train_dataloader = self.process_data(dataset_file, post_id_divisions_file, 'train')
        # self.val_dataloader = self.process_data(dataset_file, post_id_divisions_file, 'test')
        # self.test_dataloader = self.process_data(dataset_file, post_id_divisions_file, 'test')

    def ek_extra_preprocess(self, text):
        remove_words=['<allcaps>','</allcaps>','<hashtag>','</hashtag>','<elongated>','<emphasis>','<repeated>','\'','s']
        word_list=text_processor.pre_process_doc(text)
        word_list=list(filter(lambda a: a not in remove_words, word_list)) 
        sent=" ".join(word_list)
        sent = re.sub(r"[<\*>]", " ",sent)
        return sent

    def tokenize(self, sentences, padding = True, max_len = 128):
        tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
        input_ids, attention_masks, token_type_ids = [], [], []
        for sent in sentences:
            encoded_dict = tokenizer.encode_plus(sent,
                                                    add_special_tokens=True,
                                                    max_length=max_len, 
                                                    padding='max_length', 
                                                    return_attention_mask = True,
                                                    return_tensors = 'pt', 
                                                    truncation = True)
            input_ids.append(encoded_dict['input_ids'])
            attention_masks.append(encoded_dict['attention_mask'])
        
        input_ids = torch.cat(input_ids, dim=0)
        attention_masks = torch.cat(attention_masks, dim=0)

        return {'input_ids': input_ids, 'attention_masks': attention_masks}
    
    def process_data(self, data):
        sentences, labels, demographies = [], [], []
        print(len(data))

        for row in data:

            label = row['label']
            # label_oh = [0]*3
            # label_oh[label] = 1
            labels.append(label)
            

            sentence = ' '.join(row['text'])
            sentences.append(sentence)

            demography = []

            for category in categories:

                if category not in label_dict: label_dict[category] = {}
                if category not in dict_label: dict_label[category] = {}

                if row[category] not in label_dict[category]: 
                    label_dict[category][row[category]] = len(label_dict[category])
                    dict_label[category][label_dict[category][row[category]]] = row[category]
                
                demography.append(label_dict[category][row[category]])
            
            demographies.append(demography)

        inputs = self.tokenize(sentences)
        return inputs, torch.Tensor(labels), torch.Tensor(demographies)
    
    def get_dataloader(self, inputs, labels, demographies):
        data = TensorDataset(inputs['input_ids'], inputs['attention_masks'], labels, demographies)
        if self.train:
            sampler = RandomSampler(data)
        else:
            sampler = SequentialSampler(data)
        return DataLoader(data, sampler=sampler, batch_size=self.batch_size)

In [ ]:
import json
with open('./data/train_demographic.json', 'r') as f:
    train_df = json.load(f)
train_data = Dataset(train_df, train = True)
with open('./data/valid_demographic.json', 'r') as f:
    val_df = json.load(f)
val_data = Dataset(val_df)
with open('./data/test_demographic.json', 'r') as f:
    test_df = json.load(f)
test_data = Dataset(test_df)

11553
1419
1407


In [ ]:
categories_count = {category: len(label_dict[category]) for category in categories}
categories_count

{'Age': 6, 'Country': 17, 'Gender': 3, 'Race': 7, 'Religion': 7}

In [ ]:
class SC_weighted_BERT(nn.Module):
    def __init__(self, model_path, labels = False, category = 'Age', label_output = False):
        super(SC_weighted_BERT, self).__init__()
        self.bert = AutoModel.from_pretrained(model_path)
        self.dropout = nn.Dropout(0.1)
        self.use_labels = labels
        self.num_labels = categories_count[category]
        print(self.num_labels)
        if labels:
            self.classifier = nn.Linear(768 + 3, self.num_labels)
        elif label_output:
            self.classifier = nn.Linear(768, self.num_labels)
            self.classifier2 = nn.Linear(768, 3)
        else:
            self.classifier = nn.Linear(768, self.num_labels)

    def forward(self, input_ids=None, attention_mask=None, labels = None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        if self.use_labels:
            pooled_output = torch.cat([pooled_output, labels], dim = 1)
        logits = self.classifier(pooled_output)
        hate_pred = self.classifier2(pooled_output)

        outputs1 = (logits,) + outputs[2:]  # add hidden states and attention if they are here
        outputs2 = (hate_pred,) + outputs[2:]  # add hidden states and attention if they are here
        return outputs1, outputs2  # (loss), logits, (hidden_states), (attentions) for each

In [ ]:
import copy
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
 
def get_predicted(preds):
    pred_flat = np.argmax(preds, axis=1).flatten()
    return pred_flat
 
def evaluate(test_dataloader, model, category = 'Age'):
    model.eval()
    y_preds, y_test = np.array([]), np.array([])

    for batch in test_dataloader:
        b_input_ids, b_input_mask, b_labels, b_demographies = batch[0].to(device), batch[1].to(device), batch[2].to(device).long(), batch[3].to(device).long()
        with torch.no_grad():        
            ypred,_ = model(b_input_ids, b_input_mask)
        ypred = ypred[0].cpu().numpy()
        label_ids = b_demographies[:, categories_dict[category]].to('cpu').numpy()
        y_preds = np.hstack((y_preds, get_predicted(ypred)))
        y_test = np.hstack((y_test, label_ids))

    score = accuracy_score(y_test, y_preds)
    report = classification_report(y_test, y_preds)
    print(report)
    return score, y_preds, y_test
 
def train(training_dataloader, validation_dataloader, model, filepath = None, weights = None, learning_rate = 2e-5, epochs = 1, print_every = 100, category = 'Age'):
    total_steps = len(training_dataloader) * epochs
    no_decay = ['bias', 'LayerNorm.weight', 'LayerNorm.bias']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps = 1e-8)
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0, # Default value in run_glue.py
                                                num_training_steps = total_steps)
    
    best_weighted_f1 = 0
    best_model = None
    # current_epoch, best_weighted_f1 = load_metrics(filepath, model, optimizer)
    if weights == None:
        criterion = nn.CrossEntropyLoss()
    else:
        criterion = nn.CrossEntropyLoss(weight=weights)
    for epoch_i in tqdm(range(0, epochs)):
        model.train()
        for step, batch in enumerate(training_dataloader):
            b_input_ids, b_input_mask, b_labels, b_demographies = batch[0].to(device), batch[1].to(device), batch[2].to(device).long(), batch[3].to(device).long()
            # print(b_demographies[:, categories_dict[category]])
            outputs1, outputs2 = model(b_input_ids, b_input_mask)
            loss = (criterion(outputs1[0], b_demographies[:, categories_dict[category]]) + criterion(outputs2[0], b_labels))/2
 
            if step%print_every == 0:
                print(loss.item())
 
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
 
        print('### Validation Set Stats')
        weighted_f1, ypred, ytest = evaluate(validation_dataloader, model, category = category)
        print("  Macro F1: {0:.3f}".format(weighted_f1))
        if weighted_f1 > best_weighted_f1:
            best_weighted_f1 = weighted_f1
            best_model = copy.deepcopy(model)
            # save_metrics(filepath, epoch_i, model, optimizer, weighted_f1)
        
    return best_model

In [ ]:
pkl_filename = 'results_demographies_multitask.pkl'

try:
    with open(pkl_filename, 'rb') as f:
        results = pickle.load(f)
except:
    results = {}
    with open(pkl_filename, 'wb') as f:
        pickle.dump(results, f)
   

categories = ['Age', 'Country', 'Religion', 'Race', 'Gender']
class_names = ['Race'] #Gender, Age, Country, Religion, Race

for c in class_names:
    # if c in results: continue
    # else: results[c] = {}

    # model = SC_weighted_BERT(model_path, labels = False, category = c).to(device)
    # model = train(train_data.DataLoader, val_data.DataLoader, model, None, epochs = 5, category = c)
    # _, ypreds, ytest = evaluate(test_data.DataLoader, model)
    # acc = accuracy_score(ytest, ypreds)
    # f1 = f1_score(ytest, ypreds, average = 'macro')

    # with open('results_demographies.pkl', 'rb') as f:
    #     results = pickle.load(f)
    # if c not in results: results[c] = {}
    # results[c]['no_labels'] = {'f1': f1, 'acc':acc}
    # with open('results_demographies.pkl', 'wb') as f:
    #     pickle.dump(results, f)

    model = SC_weighted_BERT(model_path, category = c, label_output = True).to(device)
    model = train(train_data.DataLoader, val_data.DataLoader, model, None, epochs = 5, category = c)
    _, ypreds, ytest = evaluate(test_data.DataLoader, model, category = c)
    acc = accuracy_score(ytest, ypreds)
    f1 = f1_score(ytest, ypreds, average = 'macro')

    with open(pkl_filename, 'rb') as f:
        results = pickle.load(f)
    if c not in results: results[c] = {}
    results[c]['used_labels'] = {'f1': f1, 'acc':acc}
    with open(pkl_filename, 'wb') as f:
        pickle.dump(results, f)

    print(results)

7


  0%|          | 0/5 [00:00<?, ?it/s]

1.3720755577087402
1.0377540588378906
0.9681323766708374
1.0065374374389648
### Validation Set Stats


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       191
         1.0       0.00      0.00      0.00       131
         2.0       0.67      1.00      0.80       944
         3.0       0.00      0.00      0.00        65
         4.0       0.00      0.00      0.00        66
         5.0       0.00      0.00      0.00         9
         6.0       0.00      0.00      0.00        13

    accuracy                           0.67      1419
   macro avg       0.10      0.14      0.11      1419
weighted avg       0.44      0.67      0.53      1419

  Macro F1: 0.665


...

0.7246900796890259
0.7317758798599243
0.6237325072288513
0.7887852191925049
### Validation Set Stats
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       191
         1.0       0.00      0.00      0.00       131
         2.0       0.67      1.00      0.80       944
         3.0       0.00      0.00      0.00        65
         4.0       0.00      0.00      0.00        66
         5.0       0.00      0.00      0.00         9
         6.0       0.00      0.00      0.00        13

    accuracy                           0.67      1419
   macro avg       0.10      0.14      0.11      1419
weighted avg       0.44      0.67      0.53      1419

  Macro F1: 0.665


 80%|████████  | 4/5 [17:04<04:14, 254.95s/it]

0.581902027130127
0.6603726744651794
0.8345416188240051
0.5548633337020874
### Validation Set Stats
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       191
         1.0       0.00      0.00      0.00       131
         2.0       0.67      1.00      0.80       944
         3.0       0.00      0.00      0.00        65
         4.0       0.00      0.00      0.00        66
         5.0       0.00      0.00      0.00         9
         6.0       0.00      0.00      0.00        13

    accuracy                           0.67      1419
   macro avg       0.10      0.14      0.11      1419
weighted avg       0.44      0.67      0.53      1419

  Macro F1: 0.665


100%|██████████| 5/5 [21:23<00:00, 256.72s/it]


              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       185
         1.0       0.00      0.00      0.00       122
         2.0       0.66      1.00      0.80       931
         3.0       0.00      0.00      0.00        71
         4.0       0.00      0.00      0.00        79
         5.0       0.00      0.00      0.00         9
         6.0       0.00      0.00      0.00        10

    accuracy                           0.66      1407
   macro avg       0.09      0.14      0.11      1407
weighted avg       0.44      0.66      0.53      1407

{'Gender': {'used_labels': {'f1': 0.29755199418419903, 'acc': 0.5600568585643213}}, 'Age': {'used_labels': {'f1': 0.10975912702946833, 'acc': 0.4534470504619758}}, 'Country': {'used_labels': {'f1': 0.06122247457292386, 'acc': 0.6609808102345416}}, 'Religion': {'used_labels': {'f1': 0.10815244825845534, 'acc': 0.6090973702914001}}, 'Race': {'used_labels': {'f1': 0.11377245508982035, 'acc': 0.661691542

In [ ]:
# for c in class_names:
#     _, ypreds, ytest = evaluate(test_data.DataLoader, model, category = c)
#     acc = accuracy_score(ytest, ypreds)
#     f1 = f1_score(ytest, ypreds, average = 'macro')

#     results[c]['used_labels'] = {'f1': f1, 'acc':acc}
#     with open(pkl_filename, 'wb') as f:
#         pickle.dump(results, f)

#     print(results)

In [ ]:
# with open('results_demographies.pkl', 'rb') as f:
#     a__ = pickle.load(f)
# a__